## 1. Load Library

In [ ]:
#!pip install -U efficientnet -qq

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import * 
import os
import shutil
import json

from PIL import Image

In [ ]:
# Creating a function to streamline the Train data set   
def train_img_path(id_str):
    return os.path.join(r"train", f"{id_str}.tif")

## 2. Exploratory Data Analysis

In [ ]:
example_path = "train/f38a6374c348f90b587e046aac6079959adf3835.tif"
example_img = Image.open(example_path)
example_array = np.array(example_img)
print(f"Image Shape = {example_array.shape}")
plt.imshow(example_img)
plt.show()

In [ ]:
train_labels_df = pd.read_csv('train_labels.csv')
train_labels_df["filename"] = train_labels_df["id"].apply(train_img_path)
train_labels_df["label"] = train_labels_df["label"].astype(str)
train_labels_df.head()

In [ ]:
train_labels_df.shape

In [ ]:
train_labels_df["label"][0]

In [ ]:
set(train_labels_df['label'])

In [ ]:
plt.figure(figsize=(15,7))
ax = sns.countplot(data=train_labels_df , x=train_labels_df['label'])

plt.xlabel('classes 0=not cancerous , 1=cancerous')
plt.ylabel('Number of Recourd')
plt.title('Count of images in each class', fontsize=20)
ax.bar_label(ax.containers[0])
plt.show()

We have 220,025 images in the train data set with 2 unique labels. 0 for not cancerous and 1 for cancerous tissues.

## Label 0 = Not-Cancerous
## Label 1 = Cancerous

In [ ]:
train_labels_df['label'].value_counts(normalize = True)

In [ ]:
sample_data = np.empty((100, 96, 96, 3), dtype=np.uint8)
sample_labels = np.empty(100, dtype=np.int8)
for i in range(len(train_labels_df))[:100]:
    img_path = train_img_path(train_labels_df['id'][i])
    img = Image.open(img_path)
    sample_data[i] = np.array(img)
    sample_labels[i] = train_labels_df['label'][i]

In [ ]:
print("Non-Cancerous Images")

selected_images = np.random.choice(sample_data[sample_labels == 0].shape[0], 12, replace=False)
grid_size = int(np.ceil(np.sqrt(12)))

fig, axs = plt.subplots(grid_size, grid_size, figsize=(5, 5))

for i, ax in enumerate(axs.flatten()):
    if i < 12:
        ax.imshow(sample_data[sample_labels == 0][selected_images[i]])
        ax.axis('off') 
    else:
        fig.delaxes(ax) 

plt.tight_layout()
plt.show()

In [ ]:
print("Cancerous Images")

selected_images = np.random.choice(sample_data[sample_labels == 1].shape[0], 12, replace=False)
grid_size = int(np.ceil(np.sqrt(12)))

fig, axs = plt.subplots(grid_size, grid_size, figsize=(5, 5))

for i, ax in enumerate(axs.flatten()):
    if i < 12:
        ax.imshow(sample_data[sample_labels == 1][selected_images[i]])
        ax.axis('off') 
    else:
        fig.delaxes(ax) 

plt.tight_layout()
plt.show()

## 3. Model Designing

In this part we will perform various steps required to properly create the Resnet 50 model. We will load the data from the disk, specify train and validation data generators while creating test generator for the final submission.

We will then create a modified Resnet 50 suited for this task.

In [ ]:
test_path = "/kaggle/input/histopathologic-cancer-detection/test"
test_ids = [filename[:-4] for filename in os.listdir(test_path)]
test_filenames = [os.path.join(test_path, filename) for filename in os.listdir(test_path)]
test_df = pd.DataFrame()
test_df["id"] = test_ids
test_df["filename"] = test_filenames

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

In [ ]:
train_generator = datagen.flow_from_dataframe(
    shuffle = True,
    dataframe = train_labels_df,
    x_col = "filename",
    y_col = "label",
    target_size = (96, 96),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "binary",
    subset = "training",
    validate_filenames = False,
    seed = 10
)

validation_generator = datagen.flow_from_dataframe(
    shuffle = True,
    dataframe=train_labels_df,
    x_col = "filename",
    y_col = "label",
    target_size=(96, 96),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "binary",
    subset = "validation",
    validate_filenames = False,
    seed = 10
)

In [ ]:
test_generator = datagen.flow_from_dataframe(
    dataframe = test_df,
    x_col = "filename",
    y_col = None,
    target_size = (96, 96),
    color_mode = "rgb",
    batch_size = 64,
    shuffle = False,
    class_mode = None,
    validate_filenames = False,
    seed = 10
)

In [ ]:
train_steps = 176020//32  # 8000 images for training
val_steps = 44005//32  

Now we will create Resnet50 model.

It is to be noted that we can actually use transfer learning and import the model and its trained parameters based on imagenet from the tensorflow API. But for this project we will create a modified model.

There will be two types of blocks, identity blocks where input and output dimension remains the same, and convolutional blocks where input and output dimensions are allowed to be changed. In both blocks we will use strong skip connections.

After convolutional we will use fully collected layers with output being a dense Sigmoid unit.

In the end we will have a Resnet model with 50 layers.

In [ ]:
def identity_block(X, f, filters, training=True, initializer = random_uniform):
    """
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = tfl.Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = tfl.BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = tfl.Activation('relu')(X)
    
    ## Set the padding = 'same'
    X = tfl.Conv2D(filters = F2, kernel_size = f, strides = 1, padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = tfl.BatchNormalization(axis = 3)(X, training = training)
    X = tfl.Activation('relu')(X)


    ## Set the padding = 'valid'
    X = tfl.Conv2D(filters = F3, kernel_size = 1, strides = 1, padding = 'valid', kernel_initializer = initializer(seed = 0))(X)
    X = tfl.BatchNormalization(axis = 3)(X, training = training) 
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = tfl.Add()([X, X_shortcut])
    X = tfl.Activation('relu')(X)

    return X


In [ ]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer = glorot_uniform):
    """
    Implementation of the convolutional block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    X = tfl.Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = tfl.BatchNormalization(axis = 3)(X, training=training)
    X = tfl.Activation('relu')(X)
    
    X = tfl.Conv2D(filters = F2, kernel_size = (f,f), strides = 1, padding='same', kernel_initializer = initializer(seed=0))(X) 
    X = tfl.BatchNormalization(axis = 3)(X, training=training)
    X = tfl.Activation('relu')(X) 

    X = tfl.Conv2D(filters = F3, kernel_size = 1, strides = 1, padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = tfl.BatchNormalization(axis = 3)(X, training=training) 

    X_shortcut = tfl.Conv2D(filters = F3, kernel_size = (1,1), strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = tfl.BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    X = tfl.Add()([X, X_shortcut])
    X = tfl.Activation('relu')(X)
    
    return X

In [ ]:
def ResNet50(input_shape = (96, 96, 3)):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = tfl.Input(input_shape)

    
    # Zero-Padding
    X = tfl.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = tfl.Conv2D(96, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfl.BatchNormalization(axis = 3)(X)
    X = tfl.Activation('relu')(X)
    X = tfl.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE
    
    # Use the instructions above in order to implement all of the Stages below
    # Make sure you don't miss adding any required parameter
    
    ## Stage 3 (≈4 lines)
    # `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    
    # the 3 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4 (≈6 lines)
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    
    # the 5 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 (≈3 lines)
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    
    # the 2 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D()(X)"
    X = tfl.AveragePooling2D(pool_size = (2,2))(X)
    

    # output layer
    X = tfl.Flatten()(X)
    X = tfl.Dense(1, kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
model = ResNet50(input_shape = (96, 96, 3))
print(model.summary())

Above is the model summary. We have 23,551,681 trainable parameters. If we use more deeper network number of parameters will increase with possible increase in Accuracy or AUC. But for this model we will use resnet 50 only.

## 4. Model Deployment

In [ ]:
from IPython.display import Image
Image(url="https://wisdomml.in/wp-content/uploads/2023/03/resnet_bannner.png")

In this part we will use the model created and fit the model with Adam optimizer and loss as binary cross entropy. We have used from logits = True for better accuracy. If reader wants to avoid it they can modify the last output layer in Resnet50 above and make the Activation function as sigmoid instead of linear.

# Create F1 - Score Metric for Classification Evaluation 

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import backend as K

# def f1_score(y_true, y_pred):
#     def recall(y_true, y_pred):
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + K.epsilon())
#         return recall

#     def precision(y_true, y_pred):
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision

#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
#     f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
#     return f1


import tensorflow as tf
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    y_pred = tf.math.round(tf.math.sigmoid(y_pred))  # Convert logits to binary predictions
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)
    
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)


In [ ]:
model.compile(optimizer = tf.keras.optimizers.Nadam(learning_rate = 0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics=['accuracy', tf.keras.metrics.AUC(), f1_score])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_steps,
    validation_data = validation_generator,
    validation_steps = val_steps,
    epochs = 10
)

In [ ]:
def merge_history(hlist):
    history = {}
    for k in hlist[0].history.keys():
        history[k] = sum([h.history[k] for h in hlist], [])
    return history

def vis_training(h, start=1):
    epoch_range = range(start, len(h['loss'])+1)
    s = slice(start-1, None)

    plt.figure(figsize=[14,4])

    n = int(len(h.keys()) / 2)

    for i in range(n):
        k = list(h.keys())[i]
        plt.subplot(1,n,i+1)
        plt.plot(epoch_range, h[k][s], label='Training')
        plt.plot(epoch_range, h['val_' + k][s], label='Validation')
        plt.xlabel('Epoch'); plt.ylabel(k); plt.title(k)
        plt.grid()
        plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
history_modelrestnet = merge_history([history])
vis_training(history_modelrestnet)

It takes around 2 hours to train the model with 10 epochs and Kaggle P100 GPU. We can increase the number of epochs but only Training loss decrease for some time after 10 epochs with little change in validation loss and accuracy. In fact we can see from above that after 6th Epoch Validation Accruacy has not changed much.

So, it will not be efficient to train with more epochs but readers can do it if they have resources and time for this.

## 5. Model Evaluation

Above we have used a threshold of 0.5 which is just assumed and is totally not correct. There was no decision boundary given with the dataset so we cant use above metrics to test the performance of the model.

In order to test the model performance we need to create a submission and get the AUC we will obtain from such submission.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

F1_score = history.history['f1_score']
F1_score_valid = history.history['val_f1_score']

plt.figure(figsize=(8, 8))

# Plot Training and Validation Accuracy
plt.subplot(3, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.title('Training and Validation Accuracy')

# Plot Training and Validation Loss
plt.subplot(3, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, max(max(loss), max(val_loss))])
plt.title('Training and Validation Loss')

# Plot Training and Validation F1 Score
plt.subplot(3, 1, 3)
plt.plot(F1_score, label='Training F1 Score')
plt.plot(F1_score_valid, label='Validation F1 Score')
plt.legend(loc='lower right')
plt.ylabel('F1 Score')
plt.ylim([0, 1])
plt.title('Training and Validation F1 Score')
plt.xlabel('Epoch')

plt.tight_layout()
plt.show()

# Model 2 --> EfficientNetB0

In [ ]:
from IPython.display import Image
Image(url="https://media.licdn.com/dms/image/D4E12AQHL32j_wE3JTw/article-cover_image-shrink_720_1280/0/1669236519664?e=1727308800&v=beta&t=BM4KMl52c-TXWz0C-SCD_MbHo1_rdXaC6UWRhTNqZqo")

In [ ]:
# base_model_EfficientNetB0 = efn.EfficientNetB0(input_shape=(96,96,3), include_top=False, weights='imagenet')

# model_EfficientNetB0 = Sequential([
#     base_model_EfficientNetB0,
    
#     Flatten(),
    
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(32, activation='relu'),
#     Dropout(0.5),
#     BatchNormalization(),
#     Dense(1, activation='sigmoid')
# ])



# model_EfficientNetB0.summary()


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
# import efficientnet.keras as efn

# base_model_EfficientNetB0 = efn.EfficientNetB0(input_shape=(96, 96, 3), include_top=False, weights='imagenet')

# model_EfficientNetB0 = Sequential([
#     base_model_EfficientNetB0,
    
#     GlobalAveragePooling2D(),
    
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(32, activation='relu'),
#     Dropout(0.5),
#     BatchNormalization(),
#     Dense(1, activation='sigmoid')
# ])

# model_EfficientNetB0.summary()


In [ ]:
# model_EfficientNetB0.compile(optimizer = tf.keras.optimizers.Nadam(0.0001),
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
#               metrics=['accuracy', tf.keras.metrics.AUC(), f1_score])

In [ ]:
# h1 = model_EfficientNetB0.fit(
#     train_generator,
#     steps_per_epoch = train_steps,
#     validation_data = validation_generator,
#     validation_steps = val_steps,
#     epochs = 10
# )

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf

# Load the EfficientNetB0 model
base_model_EfficientNetB0 = EfficientNetB0(input_shape=(96, 96, 3), include_top=False, weights='imagenet')

# Add layers using Functional API
x = GlobalAveragePooling2D()(base_model_EfficientNetB0.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model_EfficientNetB0 = Model(inputs=base_model_EfficientNetB0.input, outputs=outputs)

# Compile the model
model_EfficientNetB0.compile(optimizer=tf.keras.optimizers.Nadam(0.0001),
                             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                             metrics=['accuracy', tf.keras.metrics.AUC(), f1_score])

# Print the model summary
model_EfficientNetB0.summary()


In [ ]:

# Fit the model
h1 = model_EfficientNetB0.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=validation_generator,
    validation_steps=val_steps,
    epochs=10
)


In [ ]:
history_model_B0 = merge_history([h1])
vis_training(history_model_B0)

In [ ]:
acc_B0 = [0.] + h1.history['accuracy']
val_acc_B0 = [0.] + h1.history['val_accuracy']

loss_B0 = h1.history['loss']
val_loss_B0 = h1.history['val_loss']

F1_score_B0 = h1.history['f1_score']
F1_score_valid_B0 = h1.history['val_f1_score']

plt.figure(figsize=(8, 8))

# Plot Training and Validation Accuracy
plt.subplot(3, 1, 1)
plt.plot(acc_B0, label='Training Accuracy')
plt.plot(val_acc_B0, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.title('Training and Validation Accuracy')

# Plot Training and Validation Loss
plt.subplot(3, 1, 2)
plt.plot(loss_B0, label='Training Loss')
plt.plot(val_loss_B0, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, max(max(loss_B0), max(val_loss_B0))])
plt.title('Training and Validation Loss')

# Plot Training and Validation F1 Score
plt.subplot(3, 1, 3)
plt.plot(F1_score_B0, label='Training F1 Score')
plt.plot(F1_score_valid_B0, label='Validation F1 Score')
plt.legend(loc='lower right')
plt.ylabel('F1 Score')
plt.ylim([0, 1])
plt.title('Training and Validation F1 Score')
plt.xlabel('Epoch')

plt.tight_layout()
plt.show()

# Model 3 --> Xception

In [ ]:
# import tensorflow as tf

# def f1_score(y_true, y_pred):
#     y_pred = tf.round(tf.sigmoid(y_pred))  # Convert logits to binary predictions
#     tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32), axis=0)
#     fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32), axis=0)
#     fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32), axis=0)
    
#     precision = tp / (tp + fp + tf.keras.backend.epsilon())
#     recall = tp / (tp + fn + tf.keras.backend.epsilon())
    
#     f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
#     return tf.reduce_mean(f1)



import tensorflow as tf

def f1_metric(y_true, y_pred):
    y_pred = tf.round(tf.sigmoid(y_pred))  # Convert logits to binary predictions

    tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32))
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32))

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())

    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return tf.reduce_mean(f1)


In [ ]:
from IPython.display import Image
Image(url="https://www.ismrm.org/21/program-files/TeaserSlides/abstracts/images/1945/ISMRM2021-001945_Fig6.png")

In [ ]:
# img_shape=(96, 96,3)
# base_model_Xception = tf.keras.applications.Xception(include_top= False, weights= "imagenet",
#                             input_shape= img_shape, pooling= 'max')
    
# model_Xception = Sequential([
#     base_model_Xception,
#     Flatten(),
#     Dropout(rate= 0.3),
#     Dense(128, activation= 'relu'),
#     Dropout(rate= 0.25),
#     Dense(1, activation= 'sigmoid')
# ])

# model_Xception.compile(optimizer = tf.keras.optimizers.Nadam(0.0001),
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
#               metrics=['accuracy', tf.keras.metrics.AUC(), f1_score])

# model_Xception.summary()


import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import Xception

# Define the image shape
img_shape = (96, 96, 3)

# Load the Xception base model
base_model_Xception = Xception(include_top=False, weights='imagenet', input_shape=img_shape)

# Add layers using Functional API
x = GlobalAveragePooling2D()(base_model_Xception.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model_Xception = Model(inputs=base_model_Xception.input, outputs=outputs)

# Compile the model
model_Xception.compile(optimizer=tf.keras.optimizers.Nadam(0.0001),
                       loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics=['accuracy', tf.keras.metrics.AUC(), f1_metric])

# Print the model summary
model_Xception.summary()


In [ ]:
h2 = model_Xception.fit(
    train_generator,
    steps_per_epoch = train_steps,
    validation_data = validation_generator,
    validation_steps = val_steps,
    epochs = 10
)

In [ ]:
history_model_Xception = merge_history([h2])
vis_training(history_model_Xception)

In [ ]:
acc_Xception = [0.] + h2.history['accuracy']
val_acc_Xception = [0.] + h2.history['val_accuracy']

loss_Xception = h2.history['loss']
val_loss_Xception = h2.history['val_loss']

F1_score_Xception = h2.history['f1_metric']
F1_score_valid_Xception = h2.history['val_f1_metric']

plt.figure(figsize=(8, 8))

# Plot Training and Validation Accuracy
plt.subplot(3, 1, 1)
plt.plot(acc_Xception, label='Training Accuracy')
plt.plot(val_acc_Xception, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.title('Training and Validation Accuracy')

# Plot Training and Validation Loss
plt.subplot(3, 1, 2)
plt.plot(loss_Xception, label='Training Loss')
plt.plot(val_loss_Xception, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, max(max(loss_Xception), max(val_loss_Xception))])
plt.title('Training and Validation Loss')

# Plot Training and Validation F1 Score
plt.subplot(3, 1, 3)
plt.plot(F1_score_Xception, label='Training F1 Score')
plt.plot(F1_score_valid_Xception, label='Validation F1 Score')
plt.legend(loc='lower right')
plt.ylabel('f1_metric')
plt.ylim([0, 1])
plt.title('Training and Validation F1 Score')
plt.xlabel('Epoch')

plt.tight_layout()
plt.show()

# Model 4 --> DenseNet121

In [ ]:
from IPython.display import Image
Image(url="https://www.researchgate.net/publication/357102154/figure/fig1/AS:1101806226681856@1639702483586/DenseNet121-architecture-with-three-dense-blocks-Layers-between-two-adjacent-blocks-are_W640.jpg")

In [ ]:
# from tensorflow.keras.applications import DenseNet121

# base_model_DenseNet121 = DenseNet121(input_shape=(96, 96, 3), include_top=False, weights='imagenet')

# model_DenseNet121 = Sequential([
#     base_model_DenseNet121,
    
#     Flatten(),
    
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(32, activation='relu'),
#     Dropout(0.5),
#     BatchNormalization(),
#     Dense(1, activation='sigmoid')
# ])

# # Compile the model
# model_DenseNet121.compile(optimizer = tf.keras.optimizers.Nadam(0.0001),
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
#               metrics=['accuracy', tf.keras.metrics.AUC(), f1_score])

# # Print the model summary
# model_DenseNet121.summary()


import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121

# Define the image shape
img_shape = (96, 96, 3)

# Load the DenseNet121 base model
base_model_DenseNet121 = DenseNet121(input_shape=img_shape, include_top=False, weights='imagenet')

# Add layers using Functional API
x = GlobalAveragePooling2D()(base_model_DenseNet121.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model_DenseNet121 = Model(inputs=base_model_DenseNet121.input, outputs=outputs)

# Compile the model
model_DenseNet121.compile(optimizer=tf.keras.optimizers.Nadam(0.0001),
                          loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                          metrics=['accuracy', tf.keras.metrics.AUC(), f1_score])

# Print the model summary
model_DenseNet121.summary()


In [ ]:
h3 = model_DenseNet121.fit(
    train_generator,
    steps_per_epoch = train_steps,
    validation_data = validation_generator,
    validation_steps = val_steps,
    epochs = 10
)

In [ ]:
history_model_DenseNet121 = merge_history([h3])
vis_training(history_model_DenseNet121)

In [ ]:
acc_DenseNet121 = [0.] + h3.history['accuracy']
val_acc_DenseNet121 = [0.] + h3.history['val_accuracy']

loss_DenseNet121 = h3.history['loss']
val_loss_DenseNet121 = h3.history['val_loss']

F1_score_DenseNet121 = h3.history['f1_score']
F1_score_valid_DenseNet121 = h3.history['val_f1_score']

plt.figure(figsize=(8, 8))

# Plot Training and Validation Accuracy
plt.subplot(3, 1, 1)
plt.plot(acc_DenseNet121, label='Training Accuracy')
plt.plot(val_acc_DenseNet121, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.title('Training and Validation Accuracy')

# Plot Training and Validation Loss
plt.subplot(3, 1, 2)
plt.plot(loss_DenseNet121, label='Training Loss')
plt.plot(val_loss_DenseNet121, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, max(max(loss_DenseNet121), max(val_loss_DenseNet121))])
plt.title('Training and Validation Loss')

# Plot Training and Validation F1 Score
plt.subplot(3, 1, 3)
plt.plot(F1_score_DenseNet121, label='Training F1 Score')
plt.plot(F1_score_valid_DenseNet121, label='Validation F1 Score')
plt.legend(loc='lower right')
plt.ylabel('F1 Score')
plt.ylim([0, 1])
plt.title('Training and Validation F1 Score')
plt.xlabel('Epoch')

plt.tight_layout()
plt.show()